## Распознавание речи

Ипортируем все необходимые пакеты

In [1]:
# библиотека для распознавания речи
import speech_recognition as sr

In [2]:
# библиотека для использования микрофона компьютера
import pyaudio

**Linux**:

```sudo apt-get install python-dev```  
```sudo apt-get install portaudio19-dev```  
```sudo pip3 install pyaudio```  

или какой-нибудь способ из перечисленных [здесь](https://www.raspberrypi.org/forums/viewtopic.php?t=25173 "форум") 

**Windows**:

```pip install pyaudio```

проверим, работает ли, запустив в командной строке  
```python -m speech_recognition```

In [3]:
# воспроизведение речи
from pygame import mixer
mixer.init()

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
sr.__version__

'3.8.1'

In [5]:
r = sr.Recognizer()

In [8]:
count = sr.AudioFile('1_2_3.wav')
with count as source:
    audio = r.record(source)

In [8]:
r.recognize_google(audio, language="ru_RU", show_all=False)

'раз два три'

In [10]:
mic = sr.Microphone()

In [11]:
with mic as source:
    r.adjust_for_ambient_noise(source)
    audio = r.listen(source)

In [17]:
r.recognize_google(audio, language="ru_RU", show_all=True)['alternative'][0]['transcript']

['раз', 'два', 'три']

## Синтез речи

In [76]:
from gtts import gTTS
import os

In [78]:
tts = gTTS(text='аааааа', lang="ru")
tts.save('audio.mp3')

In [17]:
mixer.music.load('audio2.mp3')
mixer.music.play()

## Домашняя работа
написать функцию, которой на вход дается предложение, которые вы наговариваете компьютеру, оно распознается и все слова в нем меняются на синонимы (используйте знания, полученные на [этом занятии](https://www.raspberrypi.org/forums/viewtopic.php?t=25173 "word2vec") ) и на выходе это новое предложение произносится.

In [6]:
import gensim
from pymorphy2  import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()
from nltk.tokenize import word_tokenize

C:\Users\user\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [18]:
m_path = 'ruscorpora_upos_skipgram_300_5_2018.vec.gz'
model = gensim.models.KeyedVectors.load_word2vec_format(m_path, binary=False)

In [19]:
model.init_sims(replace=True)

In [66]:
def counter():
    i = 0
    while True:
        yield i
        i += 1

In [82]:
cou = counter()

In [20]:
mic = sr.Microphone()

In [23]:
with mic as source:
    r.adjust_for_ambient_noise(source)
    audio = r.listen(source)

In [24]:
record = r.recognize_google(audio, language="ru_RU", show_all=True)['alternative'][0]['transcript']
record

'клёвый день иду пишу как по лыжне'

In [275]:
record = input()

три грустных друга идут


In [276]:
sent = word_tokenize(record)
sent

['три', 'грустных', 'друга', 'идут']

In [359]:
#новая версия

new_sent = []
for word in sent:
    a = 0
    w = pymorphy2_analyzer.parse(word)
    
    
    if str(w[0].tag) == 'PNCT': #отсеиваем пунктуацию
        new_sent.append(word)
        
    else:
        
        tags = str(w[0].tag).split(",")
        print(tags)
        PoS = d[w[0].tag.POS]
        mod_word = w[0].normal_form + '_' + PoS
        
        if mod_word in model:
            print('есть в модели')
            res = model.most_similar(positive=[mod_word], topn=5)
            
            for elems in res:
                t = elems[0].find('_')
                new_PoS = elems[0].replace(elems[0][:t] + '_', '')
                
                if new_PoS == PoS and a == 0:
                    slovo = elems[0].replace(PoS, '').replace('_', '')
                    
                    for tag in tags:
                        analyz_slovo = pymorphy2_analyzer.parse(slovo)
                        slovo_flec = analyz_slovo[0].inflect({tag})
                        slovo = slovo_flec[0]
                    
                    new_sent.append(slovo)
                    a = 1
            print('\n')
        else:
            print('нет в модели\n')
            new_sent.append(word)

['NUMR inan', 'accs']
нет в модели

['ADJF', 'Qual plur', 'gent']
есть в модели


C:\Users\user\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


ValueError: Unknown grammeme: Qual plur

In [355]:
s = ''
for elem in new_sent:
    s += elem
    s += ' '
print(s)

три печальный дружка пойти 


In [323]:
w = pymorphy2_analyzer.parse('ежов')
w_flec = w[0].inflect({'gent'})
w = w_flec[0]
w

'ежова'

In [124]:
j = str(next(cou))
j

'8'

In [125]:
tts = gTTS(text=s, lang="ru")
tts.save('audio'+ j +'.mp3')

In [126]:
mixer.music.load('audio' + j + '.mp3')
mixer.music.play()

In [132]:
d = {
    'ADJF': 'ADJ',
    'ADVB': 'ADV',
    'ADJS': 'ADJ',
    'NUMR': 'NUM',
    'PREP': 'ADP',
    'CONJ': 'CCONJ',
    'INFN': 'VERB',
    'VERB': 'VERB',
    'NOUN': 'NOUN',
    'NPRO': 'PRON',
    'PRTS': 'ADJ',
    'PRTF': 'ADJ',
    'INTJ': 'INTJ',
    'PROPN': 'NOUN',
    'PRCL': 'PART',
    'PNCT': 'PNCT'
}

In [106]:
##старая версия

new_sent = []
for word in sent:
    w = pymorphy2_analyzer.parse(word)
    tag = d[w[0].tag.POS]
    mod_word = w[0].normal_form + '_' + tag
    if mod_word in model:
        res = model.most_similar(positive=[mod_word], topn=1)
        for elems in res:
            elem = elems[0].replace(tag, '').replace('_', '').replace('NOUN', '').replace('PROPN', '')
            new_sent.append(elem)
    else:
        print(mod_word)
        new_sent.append(word)

всё_PART
о_ADP
тот_ADJ
же_PART


C:\Users\user\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
